Build a label dictionery and collect features for chats

In [35]:
MAX_SENT_LENGTH = 100
MAX_SENTS = 15
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2


import nltk
nltk.download('brown')
# from nltk.corpus import brown
from nltk import tokenize
from keras.preprocessing.text import Tokenizer, text_to_word_sequence

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
# tokenizer.fit_on_texts(brown.words())

[nltk_data] Downloading package brown to /home/amirgilad/nltk_data...
[nltk_data]   Package brown is already up-to-date!


Parse features of chats and adapt to model

In [33]:
import os
import json

def build_label_dict():
    d = {}
    label_path = r"/home/amirgilad/Documents/git/NLPProj/Frames-dataset/labels.txt"
    label_file = open(label_path, encoding='utf-8')
    for line in label_file:
        k, v = line.split("\t")
        d[k] = v.strip()
    return d

def gen_chat_model(label_dict):
    chat_path = r"/home/amirgilad/Documents/git/NLPProj/Frames-dataset/chats"
    chat_features = []
    avg_chat_len = 0
    for filename in os.listdir(chat_path):
        chat_file = open(os.path.join(chat_path, filename), encoding='utf-8')
        chat = json.load(chat_file)
        sents = [] # chat level docs
        if 'turns' in chat:
            for turn in chat['turns']:
#                 if 'acts_without_refs' in turn['labels'] and 'name' in turn['labels']['acts_without_refs'][0]:
#                     tokenizer.word_index[ turn['labels']['acts_without_refs'][0]['name'] ] # add type of request
#                 print(turn['labels'])
#                 tokenizer.word_index[ turn['labels']['acts'][0]['name'] ]
                if len(turn['labels']['acts']) > 0:
                    sents.append( turn['labels']['acts'][0]['name'].replace("_", " ") )
                turn['timestamp']
                avg_chat_len += 1
        chat_features.extend( sents )
    avg_chat_len /= len(chat_features)
    print("average number of sents in chat: " + str(avg_chat_len))
    return chat_features

d = build_label_dict()
chat_features = gen_chat_model(d)
tokenizer.fit_on_texts(chat_features) # fit model to labels

average number of sents in chat: 1.0069528415961306


Create vectors from features

In [44]:
import os
import json
import numpy as np 

def build_label_dict():
    d = {}
    label_path = r"/home/amirgilad/Documents/git/NLPProj/Frames-dataset/labels.txt"
    label_file = open(label_path, encoding='utf-8')
    for line in label_file:
        k, v = line.split("\t")
        d[k] = v.strip()
    return d

def compute_sum_ts(prev_ts, cur_ts, sum_ts):
    if cur_ts != 0:
        sum_ts += cur_ts - prev_ts
    return sum_ts

def gen_vectors(d):
    chat_path = r"/home/amirgilad/Documents/git/NLPProj/Frames-dataset/chats"
    chat_features = []
    labels = []
    for filename in os.listdir(chat_path):
        chat_vec = np.zeros(4)
        turns = 0
        prev_ts, sum_ts = 0,0
        first_ts = 0
        chat_file = open(os.path.join(chat_path, filename), encoding='utf-8')
        chat = json.load(chat_file)
        sents = [] # chat level docs
        if 'turns' in chat:
            for turn in chat['turns']:
#                 if len(turn['labels']['acts']) > 0:
#                     sents.append( turn['labels']['acts'][0]['name'].replace("_", " ") )
                sum_ts = compute_sum_ts(prev_ts, turn['timestamp'], sum_ts)# compute avg time between questions and responses
                prev_ts = turn['timestamp']
                turns += 1
                first_ts = turn['timestamp'] if first_ts == 0 else first_ts
        chat_vec[0] = sum_ts / turns # avg time
        chat_vec[1] = 0 # most common action
        chat_vec[2] = prev_ts - first_ts # chat total time
        chat_vec[3] = turns # number of turns
        chat_features.append( chat_vec )
        labels.append(d[filename])
        
    return np.array(chat_features), labels

In [46]:
from keras.layers import Input, Dense, merge, Embedding
from keras.models import Sequential
TRAIN_SIZE = 1000
INPUT_DIM = 4

# Generate data
d = build_label_dict()
X, Y = gen_vectors(d)

# Divide into training and test data
x_train, y_train = X[:TRAIN_SIZE], Y[:TRAIN_SIZE]
x_test, y_test = X[TRAIN_SIZE:], Y[TRAIN_SIZE:]

# Define the model
model = Sequential()
model.add(Dense(64, input_dim=INPUT_DIM, activation='relu'))
model.add(Dense(32, activation='softmax'))
model.add(Dense(16, activation='softmax'))
model.add(Dense(1, activation='sigmoid'))

# Compile and fit
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=100, batch_size=32)

# Test the model
scores = model.evaluate(x_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/100
1000/1000 [==============================] - 0s - loss: 0.6827 - acc: 0.6140     
Epoch 2/100
1000/1000 [==============================] - 0s - loss: 0.6782 - acc: 0.6140     
Epoch 3/100
1000/1000 [==============================] - 0s - loss: 0.6748 - acc: 0.6140     
Epoch 4/100
1000/1000 [==============================] - 0s - loss: 0.6725 - acc: 0.6140     
Epoch 5/100
1000/1000 [==============================] - 0s - loss: 0.6706 - acc: 0.6140     
Epoch 6/100
1000/1000 [==============================] - 0s - loss: 0.6693 - acc: 0.6140     
Epoch 7/100
1000/1000 [==============================] - 0s - loss: 0.6686 - acc: 0.6140     
Epoch 8/100
1000/1000 [==============================] - 0s - loss: 0.6682 - acc: 0.6140     
Epoch 9/100
1000/1000 [==============================] - 0s - loss: 0.6677 - acc: 0.6140     
Epoch 10/100
1000/1000 [==============================] - 0s - loss: 0.6675 - acc: 0.6140     
Epoch 11/100
1000/1000 [==============================] - 0

1000/1000 [==============================] - 0s - loss: 0.6670 - acc: 0.6140     
Epoch 87/100
1000/1000 [==============================] - 0s - loss: 0.6670 - acc: 0.6140     
Epoch 88/100
1000/1000 [==============================] - 0s - loss: 0.6670 - acc: 0.6140     
Epoch 89/100
1000/1000 [==============================] - 0s - loss: 0.6670 - acc: 0.6140     
Epoch 90/100
1000/1000 [==============================] - 0s - loss: 0.6670 - acc: 0.6140     
Epoch 91/100
1000/1000 [==============================] - 0s - loss: 0.6670 - acc: 0.6140     
Epoch 92/100
1000/1000 [==============================] - 0s - loss: 0.6670 - acc: 0.6140     
Epoch 93/100
1000/1000 [==============================] - 0s - loss: 0.6670 - acc: 0.6140     
Epoch 94/100
1000/1000 [==============================] - 0s - loss: 0.6670 - acc: 0.6140     
Epoch 95/100
1000/1000 [==============================] - 0s - loss: 0.6669 - acc: 0.6140     
Epoch 96/100
1000/1000 [==============================] - 0s - 

In [55]:
# author - Richard Liao 
# Dec 26 2016
import numpy as np
# import pandas as pd
# import cPickle
from collections import defaultdict
import re

# from bs4 import BeautifulSoup

import sys
import os

os.environ['KERAS_BACKEND']='theano'

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

MAX_SENT_LENGTH = 100
MAX_SENTS = 15
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()

# data_train = pd.read_csv('~/Testground/data/imdb/labeledTrainData.tsv', sep='\t')
# print data_train.shape

from nltk import tokenize

# reviews = []
# labels = []
# texts = []

# for idx in range(data_train.review.shape[0]):
#     text = BeautifulSoup(data_train.review[idx])
#     text = clean_str(text.get_text().encode('ascii','ignore'))
#     texts.append(text)
#     sentences = tokenize.sent_tokenize(text)
#     reviews.append(sentences)
    
#     labels.append(data_train.sentiment[idx])

# tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
# tokenizer.fit_on_texts(texts)

# data = np.zeros((len(texts), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')

# for i, sentences in enumerate(reviews):
#     for j, sent in enumerate(sentences):
#         if j< MAX_SENTS:
#             wordTokens = text_to_word_sequence(sent)
#             k=0
#             for _, word in enumerate(wordTokens):
#                 if k<MAX_SENT_LENGTH and tokenizer.word_index[word]<MAX_NB_WORDS:
#                     data[i,j,k] = tokenizer.word_index[word]
#                     k=k+1                    
                    
# word_index = tokenizer.word_index
# print('Total %s unique tokens.' % len(word_index))

# labels = to_categorical(np.asarray(labels))
# print('Shape of data tensor:', data.shape)
# print('Shape of label tensor:', labels.shape)

# indices = np.arange(data.shape[0])
# np.random.shuffle(indices)
# data = data[indices]
# labels = labels[indices]
# nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

# x_train = data[:-nb_validation_samples]
# y_train = labels[:-nb_validation_samples]
# x_val = data[-nb_validation_samples:]
# y_val = labels[-nb_validation_samples:]

# print('Number of positive and negative reviews in traing and validation set')
# # print y_train.sum(axis=0)
# # print y_val.sum(axis=0)

# GLOVE_DIR = "/ext/home/analyst/Testground/data/glove"
# embeddings_index = {}
# f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
# for line in f:
#     values = line.split()
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()

# print('Total %s word vectors.' % len(embeddings_index))

# embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
# for word, i in word_index.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         # words not found in embedding index will be all-zeros.
#         embedding_matrix[i] = embedding_vector
        
# embedding_layer = Embedding(len(word_index) + 1,
#                             EMBEDDING_DIM,
#                             weights=[embedding_matrix],
#                             input_length=MAX_SENT_LENGTH,
#                             trainable=True)

# sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
# embedded_sequences = embedding_layer(sentence_input)
# l_lstm = Bidirectional(LSTM(100))(embedded_sequences)
# sentEncoder = Model(sentence_input, l_lstm)

# review_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH), dtype='int32')
# review_encoder = TimeDistributed(sentEncoder)(review_input)
# l_lstm_sent = Bidirectional(LSTM(100))(review_encoder)
# preds = Dense(2, activation='softmax')(l_lstm_sent)
# model = Model(review_input, preds)

# model.compile(loss='categorical_crossentropy',
#               optimizer='rmsprop',
#               metrics=['acc'])

# print("model fitting - Hierachical LSTM")
# print (model.summary())
# model.fit(x_train, y_train, validation_data=(x_val, y_val),
#           nb_epoch=10, batch_size=50)

# building Hierachical Attention network
# embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
# for word, i in word_index.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         # words not found in embedding index will be all-zeros.
#         embedding_matrix[i] = embedding_vector
        
embedding_layer = Embedding(1000 + 1,
                            EMBEDDING_DIM,
                            weights=[1],
                            input_length=MAX_SENT_LENGTH,
                            trainable=True)

class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializations.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):
        eij = K.tanh(K.dot(x, self.W))
        
        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')
        
        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)

    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])

sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(GRU(100, return_sequences=True))(embedded_sequences)
l_dense = TimeDistributed(Dense(200))(l_lstm)
l_att = AttLayer()(l_dense)


sentEncoder = Model(sentence_input, l_att)

review_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(GRU(100, return_sequences=True))(review_encoder)
l_dense_sent = TimeDistributed(Dense(200))(l_lstm_sent)
l_att_sent = AttLayer()(l_dense_sent)
preds = Dense(2, activation='softmax')(l_att_sent)
model = Model(review_input, preds)

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - Hierachical attention network")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=50)

AttributeError: 'int' object has no attribute 'shape'